In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

In [2]:
# load in training dataset 

train = pd.read_csv('./train.csv')

In [3]:
# load in aggregated member data, which has data on average churn by city and registration method

members_grouped = pd.read_csv('./members_grouped.csv')

# drop duplicated field
members_grouped = members_grouped.drop('Unnamed: 0', axis=1)
print('done')

# print the first few records of the dataset
members_grouped.head()

done


,msno,city_churn_avg,registered_via_churn_avg
0,waLDQMmcOu2jLDaV1ddDkgCrB/jl6sD66Xzs0Vqax1Y=,0.082249,0.086362
1,QA7uiXy8vIbUSPOkCf9RwQ3FsT8jVq2OxDr8zqa7bRQ=,0.091076,0.086362
2,fGwBva6hikQmTJzrbz/2Ezjm5Cth5jZUNvXigKK2AFA=,0.084108,0.086362
3,mT5V8rEpa+8wuqi6x0DoVd3H5icMKkE9Prt49UlmK+4=,0.078733,0.086362
4,XaPhtGLk/5UvvOYHcONTwsnH97P4eGECeq+BARGItRw=,0.082924,0.086362


In [4]:
# load in the aggregated transaction data, which has data on payment plans, prices, renewal and cancel rates

transactions_grouped = pd.read_csv('./transactions_grouped.csv')

# drop duplicated field
transactions_grouped = transactions_grouped.drop('Unnamed: 0', axis=1)
print('done')

# print the first few records of the dataset
transactions_grouped.head()

done


,msno,avg_payment_plan_days,avg_plan_list_price,avg_actual_amount_paid,pct_is_auto_renew,pct_is_cancel,total_transactions,is_cancel,is_auto_renew,pay_method_id_churn_avg
0,+++FOrTS7ab3tIgIh8eWwX4FqRv8w/FoiOuyXsFvphY=,7.0,0.0,0.0,0.0,0.0,1,0,0,NaN
1,+++IZseRRiQS9aaSkH6cMYU6bGDcxUieAi/tH67sC5s=,410.0,1788.0,1788.0,0.0,0.0,1,0,0,NaN
2,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,30.0,99.0,99.0,1.0,0.0,4,0,4,0.031462
3,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,30.0,99.0,99.0,1.0,0.0,4,0,4,0.031462
4,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,30.0,99.0,99.0,1.0,0.0,4,0,4,0.031462


In [5]:
# load in the aggregated user log data, which has data on streaming activity 
user_logs_grouped = pd.read_csv('./user_logs_grouped.csv')
print('done')

# print the first few records of the datasets
user_logs_grouped.head()

done


,Unnamed: 0,msno,total_num_unq,total_total_secs,daily_avg_num_25,daily_avg_num_50,daily_avg_num_75,daily_avg_num_985,daily_avg_num_100,daily_avg_num_unq,daily_avg_total_secs,tier
0,0,+++FOrTS7ab3tIgIh8eWwX4FqRv8w/FoiOuyXsFvphY=,121.0,33169.777,3.000000,1.166667,0.500000,1.000000,19.500000,20.166667,5528.296167,50_75
1,1,+++IZseRRiQS9aaSkH6cMYU6bGDcxUieAi/tH67sC5s=,19063.0,5290185.401,1.223684,0.416667,0.328947,0.267544,94.513158,83.609649,23202.567548,top_75
2,2,+++dz9ZCWE2HB/47pJU82NJXQzQuZDx1Wm50YSk/kKk=,94.0,27344.668,1.500000,0.000000,0.000000,0.500000,52.500000,47.000000,13672.334000,25_50
3,3,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,964.0,256241.646,5.631579,2.605263,2.868421,4.684211,20.815789,25.368421,6743.201211,50_75
4,4,+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=,5059.0,1354749.777,1.802632,0.706140,0.640351,0.929825,22.530702,22.188596,5941.884987,top_75


In [6]:
# checking to see if negative streaming duration records were finally removed...
user_logs_grouped.describe()

,Unnamed: 0,total_num_unq,total_total_secs,daily_avg_num_25,daily_avg_num_50,daily_avg_num_75,daily_avg_num_985,daily_avg_num_100,daily_avg_num_unq,daily_avg_total_secs
count,3.012972e+06,3.012972e+06,3.012972e+06,3.012972e+06,3.012972e+06,3.012972e+06,3.012972e+06,3.012972e+06,3.012972e+06,3.012972e+06
mean,1.506486e+06,1.472707e+03,7.668756e+10,6.049276e+00,1.642763e+00,8.909188e-01,8.472906e-01,1.743210e+01,1.960839e+01,1.082001e+09
std,8.697702e+05,3.096570e+03,2.958625e+13,9.768570e+00,3.217721e+00,1.574308e+00,1.550713e+00,2.300296e+01,1.915614e+01,5.318053e+11
min,0.000000e+00,1.000000e+00,1.000000e-03,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e-03
25%,7.532428e+05,1.400000e+01,2.310949e+03,1.600000e+00,4.000000e-01,1.372549e-01,0.000000e+00,2.666667e+00,6.000000e+00,1.051909e+03
50%,1.506486e+06,1.160000e+02,2.647177e+04,3.562500e+00,1.000000e+00,6.000000e-01,5.614035e-01,1.166667e+01,1.525000e+01,3.428316e+03
75%,2.259728e+06,1.518000e+03,3.900141e+05,7.022346e+00,1.964912e+00,1.052632e+00,1.026316e+00,2.350617e+01,2.654796e+01,6.449649e+03
max,3.012971e+06,9.900700e+04,2.767012e+16,1.444500e+03,4.410000e+02,2.410000e+02,4.748737e+02,9.046167e+03,1.188444e+03,5.425513e+14


In [7]:
# begin merging the aggregated datasets -- start with merging the members data with transactions data on msno
train_master = members_grouped.merge(transactions_grouped, how='left', on='msno')
print('done')

# print the first few records of the dataset -- dataset merged correctly
train_master.head()

done


,msno,city_churn_avg,registered_via_churn_avg,avg_payment_plan_days,avg_plan_list_price,avg_actual_amount_paid,pct_is_auto_renew,pct_is_cancel,total_transactions,is_cancel,is_auto_renew,pay_method_id_churn_avg
0,waLDQMmcOu2jLDaV1ddDkgCrB/jl6sD66Xzs0Vqax1Y=,0.082249,0.086362,18.500000,74.500000,74.500000,0.000000,0.000000,2,0,0,0.196290
1,waLDQMmcOu2jLDaV1ddDkgCrB/jl6sD66Xzs0Vqax1Y=,0.082249,0.086362,18.500000,74.500000,74.500000,0.000000,0.000000,2,0,0,0.196290
2,QA7uiXy8vIbUSPOkCf9RwQ3FsT8jVq2OxDr8zqa7bRQ=,0.091076,0.086362,28.826087,143.869565,150.347826,0.956522,0.086957,23,2,22,0.036525
3,QA7uiXy8vIbUSPOkCf9RwQ3FsT8jVq2OxDr8zqa7bRQ=,0.091076,0.086362,28.826087,143.869565,150.347826,0.956522,0.086957,23,2,22,0.036525
4,QA7uiXy8vIbUSPOkCf9RwQ3FsT8jVq2OxDr8zqa7bRQ=,0.091076,0.086362,28.826087,143.869565,150.347826,0.956522,0.086957,23,2,22,0.036525


In [8]:
# Now merging the aggregated user streaming log dataset on msno
train_master = train_master.merge(user_logs_grouped, how='left', on='msno')
print('done')

# print the first few records of the dataset -- dataset merged correctly
train_master.head()

done


,msno,city_churn_avg,registered_via_churn_avg,avg_payment_plan_days,avg_plan_list_price,avg_actual_amount_paid,pct_is_auto_renew,pct_is_cancel,total_transactions,is_cancel,...,total_num_unq,total_total_secs,daily_avg_num_25,daily_avg_num_50,daily_avg_num_75,daily_avg_num_985,daily_avg_num_100,daily_avg_num_unq,daily_avg_total_secs,tier
0,waLDQMmcOu2jLDaV1ddDkgCrB/jl6sD66Xzs0Vqax1Y=,0.082249,0.086362,18.500000,74.500000,74.500000,0.000000,0.000000,2,0,...,173.0,40919.548,2.000000,0.363636,0.636364,0.272727,13.818182,15.727273,3719.958909,50_75
1,waLDQMmcOu2jLDaV1ddDkgCrB/jl6sD66Xzs0Vqax1Y=,0.082249,0.086362,18.500000,74.500000,74.500000,0.000000,0.000000,2,0,...,173.0,40919.548,2.000000,0.363636,0.636364,0.272727,13.818182,15.727273,3719.958909,50_75
2,QA7uiXy8vIbUSPOkCf9RwQ3FsT8jVq2OxDr8zqa7bRQ=,0.091076,0.086362,28.826087,143.869565,150.347826,0.956522,0.086957,23,2,...,2796.0,966921.058,0.910526,0.289474,0.210526,0.400000,19.152632,14.715789,5089.058200,top_75
3,QA7uiXy8vIbUSPOkCf9RwQ3FsT8jVq2OxDr8zqa7bRQ=,0.091076,0.086362,28.826087,143.869565,150.347826,0.956522,0.086957,23,2,...,2796.0,966921.058,0.910526,0.289474,0.210526,0.400000,19.152632,14.715789,5089.058200,top_75
4,QA7uiXy8vIbUSPOkCf9RwQ3FsT8jVq2OxDr8zqa7bRQ=,0.091076,0.086362,28.826087,143.869565,150.347826,0.956522,0.086957,23,2,...,2796.0,966921.058,0.910526,0.289474,0.210526,0.400000,19.152632,14.715789,5089.058200,top_75


In [9]:
# Now merge in the training dataset on msno
train_master = train.merge(train_master, how='left', on='msno')
print('done')

# print the first few records of the dataset -- dataset merged correctly
train_master.head()

done


,msno,is_churn,city_churn_avg,registered_via_churn_avg,avg_payment_plan_days,avg_plan_list_price,avg_actual_amount_paid,pct_is_auto_renew,pct_is_cancel,total_transactions,...,total_num_unq,total_total_secs,daily_avg_num_25,daily_avg_num_50,daily_avg_num_75,daily_avg_num_985,daily_avg_num_100,daily_avg_num_unq,daily_avg_total_secs,tier
0,waLDQMmcOu2jLDaV1ddDkgCrB/jl6sD66Xzs0Vqax1Y=,1,0.082249,0.086362,18.500000,74.500000,74.500000,0.000000,0.000000,2,...,173.0,40919.548,2.000000,0.363636,0.636364,0.272727,13.818182,15.727273,3719.958909,50_75
1,waLDQMmcOu2jLDaV1ddDkgCrB/jl6sD66Xzs0Vqax1Y=,1,0.082249,0.086362,18.500000,74.500000,74.500000,0.000000,0.000000,2,...,173.0,40919.548,2.000000,0.363636,0.636364,0.272727,13.818182,15.727273,3719.958909,50_75
2,QA7uiXy8vIbUSPOkCf9RwQ3FsT8jVq2OxDr8zqa7bRQ=,1,0.091076,0.086362,28.826087,143.869565,150.347826,0.956522,0.086957,23,...,2796.0,966921.058,0.910526,0.289474,0.210526,0.400000,19.152632,14.715789,5089.058200,top_75
3,QA7uiXy8vIbUSPOkCf9RwQ3FsT8jVq2OxDr8zqa7bRQ=,1,0.091076,0.086362,28.826087,143.869565,150.347826,0.956522,0.086957,23,...,2796.0,966921.058,0.910526,0.289474,0.210526,0.400000,19.152632,14.715789,5089.058200,top_75
4,QA7uiXy8vIbUSPOkCf9RwQ3FsT8jVq2OxDr8zqa7bRQ=,1,0.091076,0.086362,28.826087,143.869565,150.347826,0.956522,0.086957,23,...,2796.0,966921.058,0.910526,0.289474,0.210526,0.400000,19.152632,14.715789,5089.058200,top_75


In [10]:
# output the combined master training set to csv
train_master.to_csv(path_or_buf='./train_master.csv')

In [11]:
# quick description of the fields
train_master.describe()

,is_churn,city_churn_avg,registered_via_churn_avg,avg_payment_plan_days,avg_plan_list_price,avg_actual_amount_paid,pct_is_auto_renew,pct_is_cancel,total_transactions,is_cancel,...,Unnamed: 0,total_num_unq,total_total_secs,daily_avg_num_25,daily_avg_num_50,daily_avg_num_75,daily_avg_num_985,daily_avg_num_100,daily_avg_num_unq,daily_avg_total_secs
count,1.588315e+07,1.428240e+07,1.428240e+07,1.588315e+07,1.588315e+07,1.588315e+07,1.588315e+07,1.588315e+07,1.588315e+07,1.588315e+07,...,1.368956e+07,1.368956e+07,1.368956e+07,1.368956e+07,1.368956e+07,1.368956e+07,1.368956e+07,1.368956e+07,1.368956e+07,1.368956e+07
mean,4.316342e-02,6.698053e-02,6.118142e-02,2.967551e+01,1.313862e+02,1.359501e+02,9.250705e-01,1.737722e-02,2.025039e+01,3.862520e-01,...,1.506171e+06,4.068250e+03,1.511442e+11,5.762589e+00,1.450718e+00,8.895776e-01,9.682740e-01,2.592525e+01,2.594455e+01,1.229317e+09
std,2.032249e-01,1.835445e-02,3.949715e-02,1.043466e+01,4.839866e+01,4.889275e+01,2.420890e-01,3.257449e-02,6.915284e+00,6.714162e-01,...,8.695221e+05,4.428654e+03,3.772886e+13,6.040541e+00,1.372904e+00,7.452542e-01,1.033972e+00,2.252496e+01,1.724861e+01,3.812220e+11
min,0.000000e+00,4.841136e-02,3.015997e-02,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,...,3.000000e+00,1.000000e+00,7.800000e-02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,7.800000e-02
25%,0.000000e+00,4.841136e-02,3.015997e-02,2.884615e+01,1.000000e+02,9.900000e+01,1.000000e+00,0.000000e+00,1.600000e+01,0.000000e+00,...,7.534400e+05,1.076000e+03,2.667426e+05,2.478947e+00,6.805556e-01,4.605263e-01,4.673913e-01,1.305882e+01,1.505263e+01,3.656855e+03
50%,0.000000e+00,7.844498e-02,3.015997e-02,3.000000e+01,1.375385e+02,1.440000e+02,1.000000e+00,0.000000e+00,2.200000e+01,0.000000e+00,...,1.505309e+06,2.766000e+03,7.057691e+05,4.223140e+00,1.110526e+00,7.252747e-01,7.500000e-01,2.021951e+01,2.202516e+01,5.488622e+03
75%,0.000000e+00,8.424126e-02,8.636210e-02,3.000000e+01,1.432857e+02,1.490000e+02,1.000000e+00,3.703704e-02,2.600000e+01,1.000000e+00,...,2.257807e+06,5.571000e+03,1.449024e+06,7.138889e+00,1.797153e+00,1.110132e+00,1.179775e+00,3.120635e+01,3.192701e+01,8.299373e+03
max,1.000000e+00,1.008324e-01,1.825579e-01,4.500000e+02,2.000000e+03,2.000000e+03,1.000000e+00,1.000000e+00,7.100000e+01,2.000000e+01,...,3.012966e+06,9.900700e+04,2.767012e+16,1.324306e+03,1.860000e+02,1.660000e+02,9.574894e+01,6.388676e+02,4.710000e+02,2.427203e+14
